# FIMO output using D. virilis

First: 
- collapse to the gene level, compare existence of TF/gene relationships

In [1]:
import pandas as pd
import gffutils
from gffutils import pybedtools_integration
import pybedtools
from pybedtools.featurefuncs import gff2bed

### Import gffs from fimo output and change to bed format:  
5 motif databases: dmmpmm, flyFactor, flyReg, idmmpmm, & onTheFly

In [2]:
#note: score is min(1000, -10*(log10(pvalue)))
#for now I'm not going to worry about trying to get the pvalue since we're just collapsing everything to gene
dmm = pybedtools.BedTool('../motif-wf/data/fimo_dvir_02-15/motif_alignments_dmmpmm2009_dvir.gff')
dmmbed = dmm.each(gff2bed, name_field='ID').saveas().to_dataframe()
dmmbed['name'] = [x.split('-')[0] for x in dmmbed.name]

In [43]:
#add column for motif_FBgn
symbolmap = pd.read_table('/data/LCDB/lcdb-references/dmel/r6-11/gtf/dmel_r6-11.SYMBOL.csv', sep=',') 
dmm_w_motiffbgn = dmmbed.merge(symbolmap, left_on='name', right_on='SYMBOL', how='left')

In [44]:
dmm_w_motiffbgn.head()

,chrom,start,end,name,score,strand,ENSEMBL,SYMBOL
0,scaffold_13049,340807,340815,bcd,48.3,-,FBgn0000166,bcd
1,scaffold_13049,630585,630593,bcd,48.3,-,FBgn0000166,bcd
2,scaffold_13049,666194,666202,bcd,48.3,-,FBgn0000166,bcd
3,scaffold_13049,915737,915745,bcd,48.3,-,FBgn0000166,bcd
4,scaffold_13049,1103900,1103908,bcd,48.3,-,FBgn0000166,bcd


In [80]:
dmm_merge = dmm_w_motiffbgn[['chrom','start','end','ENSEMBL']].rename(columns={'ENSEMBL':'name'})

In [26]:
flyfac = pybedtools.BedTool('../motif-wf/data/fimo_dvir_02-15/motif_alignments_flyFactor_dvir.gff')
ffbed = flyfac.each(gff2bed, name_field='Name').saveas().to_dataframe()
ffbed['name'] = [x.split('_')[0] for x in ffbed.name]

In [30]:
ffbed.head()

,chrom,start,end,name,score,strand
0,scaffold_13049,53234,53242,FBgn0000014,45.8,+
1,scaffold_13049,145683,145691,FBgn0000014,45.8,+
2,scaffold_13049,149968,149976,FBgn0000014,45.8,+
3,scaffold_13049,155609,155617,FBgn0000014,45.8,+
4,scaffold_13049,181619,181627,FBgn0000014,45.8,+


In [78]:
ffmerge = ffbed[['chrom','start','end','name']]

In [33]:
flyreg = pybedtools.BedTool('../motif-wf/data/fimo_dvir_02-15/motif_alignments_flyReg_dvir.gff')
frbed = flyreg.each(gff2bed, name_field='Name').saveas().to_dataframe()
frbed['name'] = [x.split('_')[0] for x in frbed.name]

In [45]:
#add column for motif_FBgn
fr_w_motiffbgn = frbed.merge(symbolmap, left_on='name', right_on='SYMBOL', how='left')

In [46]:
fr_w_motiffbgn.head()

,chrom,start,end,name,score,strand,ENSEMBL,SYMBOL
0,scaffold_13049,53234,53242,abd-A,45.8,+,FBgn0000014,abd-A
1,scaffold_13049,145683,145691,abd-A,45.8,+,FBgn0000014,abd-A
2,scaffold_13049,149968,149976,abd-A,45.8,+,FBgn0000014,abd-A
3,scaffold_13049,155609,155617,abd-A,45.8,+,FBgn0000014,abd-A
4,scaffold_13049,181619,181627,abd-A,45.8,+,FBgn0000014,abd-A


In [82]:
fr_merge = fr_w_motiffbgn[['chrom','start','end','ENSEMBL']].rename(columns={'ENSEMBL':'name'})

In [35]:
idm = pybedtools.BedTool('../motif-wf/data/fimo_dvir_02-15/motif_alignments_idmmpmm2009_dvir.gff')
idmbed = idm.each(gff2bed, name_field='Name').saveas().to_dataframe()
idmbed['name'] = [x.split('_')[0] for x in idmbed.name]

In [47]:
idm_w_motiffbgn = idmbed.merge(symbolmap, left_on='name', right_on='SYMBOL', how='left')

In [48]:
idm_w_motiffbgn.head()

,chrom,start,end,name,score,strand,ENSEMBL,SYMBOL
0,scaffold_13049,15087,15095,abd-A,45.8,+,FBgn0000014,abd-A
1,scaffold_13049,80992,81000,abd-A,45.8,+,FBgn0000014,abd-A
2,scaffold_13049,209212,209220,abd-A,45.8,+,FBgn0000014,abd-A
3,scaffold_13049,270146,270154,abd-A,45.8,+,FBgn0000014,abd-A
4,scaffold_13049,270292,270300,abd-A,45.8,+,FBgn0000014,abd-A


In [83]:
idm_merge = idm_w_motiffbgn[['chrom','start','end','ENSEMBL']].rename(columns={'ENSEMBL':'name'})

In [ ]:
alt_otf = onthefly.each(gff2bed, name_field='Name').saveas().to_dataframe()
alt_otf['name'] = [x.split('_')[0] for x in otfbed.name]

In [ ]:
alt_otf.head()

In [ ]:
#add column for motif_FBGN
otfmap = pd.read_table('../motif-wf/data/onTheFlyMap.tsv')
y = otfmap[['name', 'FBgn']]
alt_otf_w_fbgn = alt_otf.merge(y, on='name', how='left')


In [ ]:
#apparently this is a mystery OTF... I will see if I have this and if I do can try to elucidate info from gff
update = update[update.name != 'OTF0415.1']

In [41]:
onthefly = pybedtools.BedTool('../motif-wf/data/fimo_dvir_02-15/motif_alignments_onTheFly_dvir.gff')
otfbed = onthefly.each(gff2bed, name_field='Alias').saveas().to_dataframe()

In [42]:
otfbed.head()

,chrom,start,end,name,score,strand
0,scaffold_13049,3366,3373,7UP1_DROME_B1H,41.9,-
1,scaffold_13049,4465,4472,7UP1_DROME_B1H,41.9,-
2,scaffold_13049,25387,25394,7UP1_DROME_B1H,41.9,-
3,scaffold_13049,28133,28140,7UP1_DROME_B1H,41.9,-
4,scaffold_13049,69626,69633,7UP1_DROME_B1H,41.9,-


In [55]:
otfbed['name'] = otfbed.name.astype(str, copy=False)

In [56]:
otfbed['name'] = [x.split('_')[0] for x in otfbed.name]

In [57]:
otfbed.head()

,chrom,start,end,name,score,strand
0,scaffold_13049,3366,3373,7UP1,41.9,-
1,scaffold_13049,4465,4472,7UP1,41.9,-
2,scaffold_13049,25387,25394,7UP1,41.9,-
3,scaffold_13049,28133,28140,7UP1,41.9,-
4,scaffold_13049,69626,69633,7UP1,41.9,-


In [67]:
lower_case = []  
for x in otfbed.name:
    if x.startswith('FBpp0'):
        lower_case.append(x)
    else:
        lower_case.append(x.lower())

In [71]:
otfbed['lower'] = lower_case

In [73]:
otf_w_motiffbgn = otfbed.merge(symbolmap, left_on='lower', right_on='SYMBOL', how='left')

In [84]:
otf_w_motiffbgn[otf_w_motiffbgn.ENSEMBL.isnull()].lower.unique()

array(['7up1', 'a0jq60', 'a1a6r5', 'a1z858', 'a1z8f4', 'a1za01', 'a1zb05',
       'a1zbv6', 'a2vef7', 'a4ij65', 'a4ij80', 'a4uz51', 'a4v1y6',
       'a4v1y7', 'a4v304', 'a4v334', 'a4vck2', 'a7vjj6', 'a8dy91',
       'a8dys7', 'a8dyz8', 'a8juv3', 'a8juy5', 'a8vem3', 'a8wdw9',
       'a8whe9', 'a8whl0', 'a9ygg0', 'a9yhi4', 'a9yik2', 'a9yik4',
       'a9yik9', 'abda', 'abdb', 'abru', 'adf1', 'aef1', 'antp', 'apte',
       'arnt', 'ast3', 'ast5', 'ast8', 'b3dn74', 'b4yw46', 'b4yxm1',
       'b6idr1', 'b6vqa1', 'b7z003', 'b7z056', 'b7z0c4', 'b7z0k0',
       'b7z0s3', 'b7z0s4', 'b7z0u8', 'b7z0y0', 'nan', 'barh1', 'barh2',
       'brc1', 'brc4', 'c0lrf2', 'c0lrg8', 'c0lrh3', 'c0lrh4', 'c0lrj9',
       'c0lrk7', 'c0lrl8', 'c0lrm0', 'c0lrm1', 'c0mk07', 'c4jc86', 'cebp',
       'cf2', 'clock', 'creba', 'cut', 'cycl', 'da', 'dei', 'dfd', 'dich',
       'disc', 'dll', 'dors', 'dri', 'e74ea', 'e74eb', 'e75ba', 'egon',
       'elf1', 'elg', 'esca', 'esm3', 'esm5', 'esm7', 'esm8', 'esmc',
       'esm

In [88]:
fix_blanks = otf_w_motiffbgn.copy()

In [ ]:
fix_blanks.loc[fix_blanks.lower == 'abda', 'ENSEMBL'] = 'FBgn0000014'
fix_blanks.loc[fix_blanks.lower == 'abdb', 'ENSEMBL'] = 'FBgn0000015'
fix_blanks.loc[fix_blanks.lower == 'abdb', 'ENSEMBL'] = 'FBgn0000015'

In [ ]:
otf_merge = otf_w_motiffbgn[['chrom','start','end',]]

In [ ]:
#put all together
#drop duplicates
#target gene intersect
#collapse